In [1]:
# Ελεγχος version pyspark .. Θέλουμε το 3.4.1
import pyspark
print(pyspark.__version__)

3.4.1


In [2]:
# add java to path .. δεν μπορουσα να βρω αλλον τροπο να τρέξει η spark.
# (θελουμε την Java 17 επειδη μονο η Java 8 , Java 11 και Java 17 δουλευει με το pyspark)
import os
os.environ["JAVA_HOME"] = "C:\\Program Files\\Java\\jdk-17"
os.environ["PATH"] += ";C:\\Program Files\\Java\\jdk-17\\bin"

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, avg, format_number
from pyspark.sql.types import FloatType
import pandas as pd

Παραδοχή : Χρησιμοποιήσαμε το pandas για να διαβασουμε τα δεδομένα και τα μετατρέψαμε σε csv και μετα χρησιμοποιήσαμε την spark επειδή δεν μας δούλευε το spark-excel

In [4]:
# Διαβάζομε τα δεδομένα

tour_occ_ninat = "data/tour_occ_ninat.xlsx"

excel_data = pd.read_excel(tour_occ_ninat, header=8)

In [5]:
# Κάνω το excel σε csv για να το βαλω στην spark , επειδη δεν μου δουλευε το spark-excel

excel_data.to_csv("data/excel_data.csv", index=False)

In [6]:
# Φτιάχνουμε ενα spark session
spark = SparkSession.builder.appName("EurostatAnalysis").getOrCreate()

In [7]:
# Διαβαζουμε το csv με την spark
spark_df = spark.read.option("header", "true").csv("data/excel_data.csv")

In [8]:
# Ενα print για να δουμε οτι διαβασαμε σωστα τα δεδομενα
spark_df.show(truncate=False)

+------------------------------------------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|GEO/TIME                                        |2006     |2007     |2008     |2009     |2010     |2011     |2012     |2013     |2014     |2015     |
+------------------------------------------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|Belgium                                         |16039090 |16271311 |16360702 |15451017 |16169676 |16723867 |16432646 |16511721 |17068872 |18852087 |
|Bulgaria                                        |11944694 |12006786 |11791454 |9460922  |10547112 |12426723 |13451440 |14370426 |14077798 |13352281 |
|Czech Republic                                  |20090348 |20610186 |19987022 |17746893 |18365947 |19424839 |21793985 |22144896 |22110112 |23286515 |
|Denmark                                         |9453026  |9327579  |8918197  |8299403  |8981

In [9]:
# Αντικαθηστώ το ":" με 0 για να βγουν σωστά οι μέση όροι
# και οποιοδήποτε αλλο string γινεται 0
# και μετατρέπω τα strings σε float για μετεπιτα υπολογισμούς

year_columns = [col for col in spark_df.columns if col.startswith("20")]
for year_col in year_columns:
    spark_df = spark_df.withColumn(
        year_col,
        when(col(year_col).cast(FloatType()).isNotNull(), col(year_col).cast(FloatType())).otherwise(0.0)
    )

# Ερωτημα 3.1 

In [10]:
# Βρίσκω τον μέσο όρο των στηλών για κάθε χρονο
year_columns = [str(year) for year in range(2006, 2016)]
average_columns = [col(column) for column in year_columns]
average_column = sum(average_columns) / len(average_columns)
result = spark_df.select("GEO/TIME", average_column.alias("AverageTourists")) \
    .groupBy("GEO/TIME").avg("AverageTourists")

In [11]:
result_formatted = result.withColumn("Μέσος Όρος Διανυκτερεύσεων", format_number(col("avg(AverageTourists)"), 2))
result_formatted = result_formatted.drop("avg(AverageTourists)")
result_formatted.show(truncate=False)

+------------------------------------------+--------------------------+
|GEO/TIME                                  |Μέσος Όρος Διανυκτερεύσεων|
+------------------------------------------+--------------------------+
|Sweden                                    |11,552,614.60             |
|Turkey                                    |28,357,565.60             |
|France                                    |118,241,260.00            |
|Greece                                    |60,419,943.60             |
|Slovakia                                  |4,331,929.40              |
|Belgium                                   |16,588,099.00             |
|Finland                                   |5,412,151.90              |
|Malta                                     |7,618,663.50              |
|Croatia                                   |44,565,893.20             |
|Italy                                     |172,808,371.20            |
|Lithuania                                 |2,123,205.30        

# Ερωτημα 3.2

In [13]:
def compare_tourist_counts(countries):
    result = {country: [] for country in countries}

    for country in countries:
        country_df = spark_df.filter(col("GEO/TIME") == country).select(*year_columns).collect()[0]
        greece_df = spark_df.filter(col("GEO/TIME") == "Greece").select(*year_columns).collect()[0]

        for idx, year_col in enumerate(year_columns):
            year_val = int(country_df[idx])
            greece_val = int(greece_df[idx])

            if year_val < greece_val:
                result[country].append(int(year_col))

    return result

In [14]:
# Έβαλα τις πρώτες 5 χώρες
selected_countries = ["Belgium", "Bulgaria", "Czech Republic", "Denmark", "Germany (until 1990 former territory of the FRG)"]
result = compare_tourist_counts(selected_countries)

for country, years in result.items():
    if len(years) > 0:
        print(f"Για {len(years)} χρόνια, η Ελλάδα είχε υψηλότερο αριθμό διανυκτερεύσεων από τη χώρα '{country}', Οι συγκεκριμένες χρονιές: {years}")
    else:
        print(f"Η Ελλάδα δεν είχε ποτέ υψηλότερο αριθμό διανυκτερεύσεων από τη χώρα '{country}'")

Για 10 χρόνια, η Ελλάδα είχε υψηλότερο αριθμό διανυκτερεύσεων από τη χώρα 'Belgium', Οι συγκεκριμένες χρονιές: [2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]
Για 10 χρόνια, η Ελλάδα είχε υψηλότερο αριθμό διανυκτερεύσεων από τη χώρα 'Bulgaria', Οι συγκεκριμένες χρονιές: [2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]
Για 10 χρόνια, η Ελλάδα είχε υψηλότερο αριθμό διανυκτερεύσεων από τη χώρα 'Czech Republic', Οι συγκεκριμένες χρονιές: [2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]
Για 10 χρόνια, η Ελλάδα είχε υψηλότερο αριθμό διανυκτερεύσεων από τη χώρα 'Denmark', Οι συγκεκριμένες χρονιές: [2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]
Για 2 χρόνια, η Ελλάδα είχε υψηλότερο αριθμό διανυκτερεύσεων από τη χώρα 'Germany (until 1990 former territory of the FRG)', Οι συγκεκριμένες χρονιές: [2009, 2011]


# Ερώτημα 3.3

In [20]:
def find_countries_with_max_tourists_per_year():
    result = {}

    for year_col in year_columns:
        max_country_row = spark_df.select("GEO/TIME", year_col).orderBy(col(year_col).desc()).limit(1).collect()[0]
        max_country = max_country_row["GEO/TIME"]
        max_value = max_country_row[year_col]

        result[year_col] = {"GEO/TIME": max_country, "value": max_value}

    return result

In [21]:
result = find_countries_with_max_tourists_per_year()

for year_col, data in result.items():
    print(f"Το {year_col} η χώρα με τον μεγαλύτερο αριθμό διανυκτερεύσεων ήταν η '{data['GEO/TIME']}' με τιμή {data['value']}")


Το 2006 η χώρα με τον μεγαλύτερο αριθμό διανυκτερεύσεων ήταν η 'Spain' με τιμή 224518080.0
Το 2007 η χώρα με τον μεγαλύτερο αριθμό διανυκτερεύσεων ήταν η 'Spain' με τιμή 225450240.0
Το 2008 η χώρα με τον μεγαλύτερο αριθμό διανυκτερεύσεων ήταν η 'Spain' με τιμή 223756224.0
Το 2009 η χώρα με τον μεγαλύτερο αριθμό διανυκτερεύσεων ήταν η 'Spain' με τιμή 200551728.0
Το 2010 η χώρα με τον μεγαλύτερο αριθμό διανυκτερεύσεων ήταν η 'Spain' με τιμή 213349648.0
Το 2011 η χώρα με τον μεγαλύτερο αριθμό διανυκτερεύσεων ήταν η 'Spain' με τιμή 239369168.0
Το 2012 η χώρα με τον μεγαλύτερο αριθμό διανυκτερεύσεων ήταν η 'Spain' με τιμή 243389008.0
Το 2013 η χώρα με τον μεγαλύτερο αριθμό διανυκτερεύσεων ήταν η 'Spain' με τιμή 252447760.0
Το 2014 η χώρα με τον μεγαλύτερο αριθμό διανυκτερεύσεων ήταν η 'Spain' με τιμή 259635792.0
Το 2015 η χώρα με τον μεγαλύτερο αριθμό διανυκτερεύσεων ήταν η 'Spain' με τιμή 269418112.0


# Ερώτημα 3.4

In [ ]:
def find_years_with_min_tourists_per_country():
    result = {}

    for country in spark_df.select("GEO/TIME").rdd.flatMap(lambda x: x).collect():
        country_df = spark_df.filter(col("GEO/TIME") == country)
        min_values = country_df.select(*year_columns).collect()[0]
        min_year_idx = min_values.index(min(min_values))
        min_year_col = year_columns[min_year_idx]
        min_value = min_values[min_year_idx]

        if country not in result:
            result[country] = {"year": min_year_col, "value": min_value}
        else:
            if min_value < result[country]["value"]:
                result[country] = {"year": min_year_col, "value": min_value}

    return result

In [ ]:
result = find_years_with_min_tourists_per_country()

for country, data in result.items():
    print(f"Για τη χώρα '{country}', ο μικρότερος αριθμός διανυκτερεύσεων ήταν το {data['year']} με τιμή {data['value']}")
